# `Automated script to download Sentinel-1 images using Alaska Satellite Facility API`


# Import libraries and packages

In [1]:
!apt-get install -y gdal-bin python3-gdal
!pip install asf_search
!pip install shapely
!pip install rasterio

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  python3-numpy
Suggested packages:
  libgdal-grass python-numpy-doc python3-dev python3-pytest
The following NEW packages will be installed:
  gdal-bin python3-gdal python3-numpy
0 upgraded, 3 newly installed, 0 to remove and 2 not upgraded.
Need to get 5,168 kB of archives.
After this operation, 25.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 python3-numpy amd64 1:1.21.5-1ubuntu22.04.1 [3,467 kB]
Get:2 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy/main amd64 python3-gdal amd64 3.8.4+dfsg-1~jammy0 [1,095 kB]
Get:3 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy/main amd64 gdal-bin amd64 3.8.4+dfsg-1~jammy0 [605 kB]
Fetched 5,168 kB in 5s (1,105 kB/s)
Selecting previously unselected package python3-numpy.
(Reading database ... 117540 files and directo

In [2]:
import asf_search as asf
from shapely.geometry import box, mapping
from datetime import datetime, timezone
import os
import zipfile
import rasterio
import rasterio.mask
from google.colab import files
import getpass
import pyproj
from shapely.ops import transform
from scipy.signal import convolve2d
import numpy as np
from rasterio import shutil as rio_shutil
from scipy.ndimage import uniform_filter

# Authenticate with Earthdata NASA

In [4]:
# --- Prompt for Earthdata credentials securely ---
username = input("Enter your Earthdata username: ")
password = getpass.getpass("Enter your Earthdata password: ")

# --- Authenticate with Earthdata credentials ---
session = asf.ASFSession().auth_with_creds(username, password)

Enter your Earthdata username: Carlosmendez1997
Enter your Earthdata password: ··········


# Define Area of Interest (AOI)

In [5]:
# --- Define bounding box coordinates ---
xmin, ymin, xmax, ymax = -68.803909, -11.033737, -68.725933, -10.979773

# --- Create shapely polygon and convert to WKT ---
bbox_geom = box(xmin, ymin, xmax, ymax)

# Define search parameters

In [6]:
# --- Define search parameters ---
platform = "Sentinel-1A" # Satellite platform
polarizations = ["VV", "VH"] # Polarizations to include
beam_mode = "IW" # Beam mode
processing_level = "GRD" # Processing level
orbit_direction = "DESCENDING"
relative_orbit = 127 # Relative orbit number

# Set date range

In [7]:
# --- Date range ---
start_date = "2026-01-01" # Start date for search
end_date = datetime.now(timezone.utc).strftime("%Y-%m-%d") # Current date as end

# Perform Search of S1

In [8]:
# --- Perform search ---
results = asf.search(
    platform=platform,
    processingLevel=processing_level,
    beamMode=beam_mode,
    polarization=polarizations,
    relativeOrbit=relative_orbit,
    start=start_date,
    end=end_date,
    intersectsWith=bbox_geom.wkt
)

# --- Print number of scenes found ---
print(f"Found {len(results)} Sentinel-1 scenes")

Found 16 Sentinel-1 scenes


# Create folder to dowload data

In [9]:
# --- Create folder for downloads ---
os.makedirs("ASF_Sentinel1", exist_ok=True)
# --- Download results into folder ---
results.download(path="ASF_Sentinel1", session=session)

/usr/local/lib/python3.12/dist-packages/asf_search/download/download.py:66: UserWarning: File already exists, skipping download: ASF_Sentinel1/OPERA_L2_RTC-S1_T127-271782-IW1_20260207T101341Z_20260207T222557Z_S1C_30_v1.0.h5
  warnings.warn(f'File already exists, skipping download: {os.path.join(path, filename)}')
/usr/local/lib/python3.12/dist-packages/asf_search/download/download.py:66: UserWarning: File already exists, skipping download: ASF_Sentinel1/OPERA_L2_RTC-S1_T127-271782-IW1_20260126T101341Z_20260126T191435Z_S1C_30_v1.0.h5
  warnings.warn(f'File already exists, skipping download: {os.path.join(path, filename)}')
/usr/local/lib/python3.12/dist-packages/asf_search/download/download.py:66: UserWarning: File already exists, skipping download: ASF_Sentinel1/OPERA_L2_RTC-S1_T127-271782-IW1_20260108T101442Z_20260108T164159Z_S1A_30_v1.0.h5
  warnings.warn(f'File already exists, skipping download: {os.path.join(path, filename)}')
/usr/local/lib/python3.12/dist-packages/asf_search/down

# Create output folder for GeoTIFFs

In [10]:
# --- Create output folder for GeoTIFFs ---
output_dir = "ASF_Sentinel1/GeoTIFFs"
os.makedirs(output_dir, exist_ok=True)

# Additional processing S1 images

## Function to assign CRS if missing

In [12]:
# --- Function to assign CRS if missing ---
def assign_crs(filepath, epsg="EPSG:4326"):
    subprocess.run(["gdal_edit.py", "-a_srs", epsg, filepath])
    print(f"Assigned CRS {epsg} to {filepath}")

## Function to clip raster

In [14]:
# --- Function to clip raster ---
def clip_raster(src_path, out_path, bbox_geom):
    with rasterio.open(src_path) as src:
        # If CRS is missing, force EPSG:4326
        crs = src.crs if src.crs is not None else rasterio.crs.CRS.from_epsg(4326)
        # Reproject AOI to raster CRS
        project = pyproj.Transformer.from_crs("EPSG:4326", crs, always_xy=True).transform
        bbox_geom_proj = transform(project, bbox_geom)
        geojson_geom = [mapping(bbox_geom_proj)]
        try:
            clipped_data, clipped_transform = rasterio.mask.mask(src, geojson_geom, crop=True)
            profile = src.profile
            profile.update({
                "height": clipped_data.shape[1],
                "width": clipped_data.shape[2],
                "transform": clipped_transform,
                "crs": crs
            })
            with rasterio.open(out_path, "w", **profile) as dst:
                dst.write(clipped_data)
            print(f"Saved clipped raster: {out_path}")
            return True  # ✅ Success
        except ValueError:
            print(f"Skipped {src_path}, AOI does not overlap raster.")
            return False  # ❌ Failure

## Lee speckle filter

In [12]:
# --- Lee speckle filter ---
def lee_filter(src_path, out_path, size=7):
    with rasterio.open(src_path) as src:
        band = src.read(1).astype(np.float32)

        # Local mean and variance
        mean = uniform_filter(band, size=size)
        mean_sq = uniform_filter(band**2, size=size)
        variance = mean_sq - mean**2

        # Global noise variance
        overall_variance = np.var(band)

        # Adaptive weight
        weight = variance / (variance + overall_variance)
        filtered = mean + weight * (band - mean)

        # Save filtered raster
        profile = src.profile
        with rasterio.open(out_path, "w", **profile) as dst:
            dst.write(filtered.astype(profile["dtype"]), 1)

        print(f"Saved Lee-filtered raster: {out_path}")

## Process only .tiff files

In [15]:
# --- Process only .tiff files ---
for root, dirs, files_in_dir in os.walk("ASF_Sentinel1"):
    for f in files_in_dir:
        if f.endswith(".tiff") and ("VV" in f or "VH" in f):
            src_path = os.path.join(root, f)
            try:
                date_str = f.split("_")[3][:8]
            except Exception:
                date_str = "unknown"
            pol = "VV" if "VV" in f else "VH"
            out_name = f"S1_{pol}_{date_str}.tif"
            out_path = os.path.join(output_dir, out_name)

            # Clip raster
            success = clip_raster(src_path, out_path, bbox_geom)

            # Apply Lee filter only if clipping succeeded
            if success and os.path.exists(out_path):
                lee_path = out_path.replace(".tif", "_lee.tif")
                lee_filter(out_path, lee_path)


Skipped ASF_Sentinel1/S1_271782_IW1_20260126T101341_VV_8DCC-BURST.tiff, AOI does not overlap raster.
Skipped ASF_Sentinel1/S1_271782_IW1_20260108T101442_VH_FFB6-BURST.tiff, AOI does not overlap raster.
Skipped ASF_Sentinel1/S1_271782_IW1_20260102T101342_VV_CB95-BURST.tiff, AOI does not overlap raster.
Skipped ASF_Sentinel1/S1_271782_IW1_20260207T101341_VH_84BC-BURST.tiff, AOI does not overlap raster.
Skipped ASF_Sentinel1/S1_271782_IW1_20260126T101341_VH_8DCC-BURST.tiff, AOI does not overlap raster.
Skipped ASF_Sentinel1/S1_271782_IW1_20260102T101342_VH_CB95-BURST.tiff, AOI does not overlap raster.
Skipped ASF_Sentinel1/S1_271782_IW1_20260108T101442_VV_FFB6-BURST.tiff, AOI does not overlap raster.
Skipped ASF_Sentinel1/S1_271782_IW1_20260207T101341_VV_84BC-BURST.tiff, AOI does not overlap raster.


# Compress all GeoTIFFs into a single ZIP

In [ ]:
# --- Compress all GeoTIFFs into a single ZIP ---
zip_path = "ASF_Sentinel1/Sentinel1_GeoTIFFs.zip"
with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for file in os.listdir(output_dir):
        file_path = os.path.join(output_dir, file)
        zipf.write(file_path, arcname=file)

print(f"✅ Compressed file ready: {zip_path}")

✅ Compressed file ready: ASF_Sentinel1/Sentinel1_GeoTIFFs.zip


# Download ZIP to local machine

In [ ]:
# --- Download ZIP to local machine ---
files.download(zip_path)

AttributeError: 'list' object has no attribute 'download'